In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\DeepLearningProject\\DeepLearningProject-1\\Chicken-disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\DeepLearningProject\\DeepLearningProject-1\\Chicken-disease'

In [5]:
import yaml

# Load the YAML configuration file
with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the correct key 'source_name' from the configuration
try:
    source_url = config['data_ingestion']['source_name']
    print("Source URL:", source_url)
except KeyError:
    print("Key 'source_name' not found in the configuration file.")


Key 'source_name' not found in the configuration file.


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
from src.Chicken_disease.constants import *
from src.Chicken_disease.utils.common import read_yaml, create_directories

In [8]:
# configuration
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config

In [9]:
# update components
import os
import urllib.request as request
import zipfile
from src.Chicken_disease import logger
from src.Chicken_disease.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extraction completed successfully")

In [11]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2024-03-20 21:03:58,210: INFO: common]: yaml file: config\config.yaml loaded successfully
[2024-03-20 21:03:58,219: INFO: common]: yaml file: params.yaml loaded successfully
[2024-03-20 21:03:58,223: INFO: common]: created directory at: artifacts
[2024-03-20 21:03:58,227: INFO: common]: created directory at: artifacts/data_ingestion
[2024-03-20 21:04:09,913: INFO: 1119586589]: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D918:2D62DB:218172:2A9DDA:65FB01E9
Accept-Ranges: bytes
Date: Wed, 20 Mar 2024 15:34:02 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100030-HYD
X-Cach